In [1]:
import wandb
# wandb.login()

In [2]:
# DNE\Scripts\Activate

import os, re, csv, random
from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np
import warnings
import transformers, datasets, logging
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", category=UserWarning, module="librosa")
warnings.filterwarnings("ignore", category=FutureWarning, module="librosa")
warnings.filterwarnings(
    "ignore", 
    message="Support for mismatched key_padding_mask and attn_mask is deprecated"
)
warnings.filterwarnings('ignore')
transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()
logging.getLogger("transformers").setLevel(logging.ERROR)
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, roc_auc_score, average_precision_score,
    classification_report
)

import librosa

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Device:", DEVICE, "-", torch.cuda.get_device_name(0))
else:
    DEVICE = torch.device("cpu")
    print("Device:", DEVICE)


Device: cuda - NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
import os, pathlib

import os
os.environ["HF_HOME"] = r"D:\\AIP491\\huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = r"D:\\AIP491\\huggingface_cache\\datasets"
os.environ["TRANSFORMERS_CACHE"] = r"D:\\AIP491\\huggingface_cache\\models"
os.environ["HF_HUB_CACHE"] = r"D:\\AIP491\\huggingface_cache\\hub"

os.environ["TMPDIR"] = os.environ["TEMP"] = os.environ["TMP"] = r"D:/AIP491/.tmp"

for p in [os.environ["HF_HOME"], os.environ["HF_DATASETS_CACHE"],
          os.environ["HF_HUB_CACHE"], os.environ["TRANSFORMERS_CACHE"],
          os.environ["TMPDIR"]]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

print("Cache chuyển sang ổ D:")
print(os.environ["HF_HOME"])


Cache chuyển sang ổ D:
D:\\AIP491\\huggingface_cache


In [4]:
from datasets import load_dataset

DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

dataset = load_dataset(
    "csv",
    data_files={
        "train":      "D:\\AIP491\\DNE\\csv\\DNE_train.csv",
        "validation": "D:\\AIP491\\DNE\\csv\\DNE_val.csv",
        "test":       "D:\\AIP491\\DNE\\csv\\DNE_test.csv"
    },
    cache_dir=DATASETS_CACHE,
)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['path', 'label'],
        num_rows: 18935
    })
    validation: Dataset({
        features: ['path', 'label'],
        num_rows: 3025
    })
    test: Dataset({
        features: ['path', 'label'],
        num_rows: 4525
    })
})


In [5]:
def encode_label(example):
    lbl = str(example["label"]).strip().lower()
    return {"label": 1 if lbl in ["pos", "positive", "1"] else 0}

dataset = dataset.map(encode_label)



Map:   0%|          | 0/18935 [00:00<?, ? examples/s]

Map:   0%|          | 0/3025 [00:00<?, ? examples/s]

Map:   0%|          | 0/4525 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoFeatureExtractor
import numpy as np, soundfile as sf, librosa, datasets

datasets.config.TORCHCODEC_AVAILABLE = False
MODEL_NAME = "facebook/wav2vec2-xls-r-300m"
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME, cache_dir=os.environ["TRANSFORMERS_CACHE"])

SAMPLING_RATE = 16000
MAX_SECONDS = 8
MAX_LEN = SAMPLING_RATE * MAX_SECONDS

def preprocess(batch):
    audio_path = batch["path"]
    try:
        x, sr = sf.read(audio_path)
        if x.ndim > 1:
            x = x.mean(axis=1)
        if sr != SAMPLING_RATE:
            x = librosa.resample(x, orig_sr=sr, target_sr=SAMPLING_RATE)
        x = x[:MAX_LEN].astype(np.float32)
        features = feature_extractor(
            x,
            sampling_rate=SAMPLING_RATE,
            padding="max_length",
            max_length=MAX_LEN,
            truncation=True,
            return_tensors="np"
        )

        return {
            "input_values": features["input_values"][0],
            "attention_mask": features["attention_mask"][0],
            "labels": int(batch["label"]),
        }

    except Exception as e:
        print(f"Lỗi đọc {audio_path}: {e}")

        dummy = np.zeros(MAX_LEN, dtype=np.float32)

        features = feature_extractor(
            dummy,
            sampling_rate=SAMPLING_RATE,
            padding="max_length",
            max_length=MAX_LEN,     
            truncation=True,
            return_tensors="np"
        )

        return {
            "input_values": features["input_values"][0],
            "attention_mask": features["attention_mask"][0],
            "labels": int(batch["label"]),
        }

dataset = dataset.map(preprocess, load_from_cache_file=False)
dataset = dataset.with_format("torch")

print("Done preprocessing.")


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Map:   0%|          | 0/18935 [00:00<?, ? examples/s]

Lỗi đọc D:\\AIP491\\DNE\\data\\CREMA_D\\audio\\1051_WSI_SAD_XX.wav: Error opening 'D:\\\\AIP491\\\\DNE\\\\data\\\\CREMA_D\\\\audio\\\\1051_WSI_SAD_XX.wav': Format not recognised.


Map:   0%|          | 0/3025 [00:00<?, ? examples/s]

Map:   0%|          | 0/4525 [00:00<?, ? examples/s]

Lỗi đọc D:\\AIP491\\DNE\\data\\CREMA_D\\audio\\1021_ITH_SAD_XX.wav: Error opening 'D:\\\\AIP491\\\\DNE\\\\data\\\\CREMA_D\\\\audio\\\\1021_ITH_SAD_XX.wav': Format not recognised.
Done preprocessing.


In [7]:
from transformers import AutoModelForAudioClassification
import evaluate
import torch

model = AutoModelForAudioClassification.from_pretrained(
    MODEL_NAME,
    cache_dir=os.environ["TRANSFORMERS_CACHE"],
    num_labels=2
)

acc_metric = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels)["precision"],
        "recall": rec_metric.compute(predictions=preds, references=labels)["recall"],
        "f1": f1_metric.compute(predictions=preds, references=labels)["f1"],
    }


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

In [8]:
import os, time, random
import wandb
import torch
from transformers import TrainerCallback

class WandbEpochLogger(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.gpu_name = "CPU"
        self.gpu_mem_total = 0
        if torch.cuda.is_available():
            self.gpu_name = torch.cuda.get_device_name(0)
            mem_info = torch.cuda.mem_get_info(device=0)
            self.gpu_mem_total = mem_info[1] / (1024**3)

    def get_gpu_stats(self):
        if torch.cuda.is_available():
            mem_info = torch.cuda.mem_get_info(device=0)
            mem_free = mem_info[0] / (1024**3)
            mem_total = mem_info[1] / (1024**3)
            mem_used = mem_total - mem_free
            usage_pct = mem_used / mem_total * 100
            return mem_used, mem_total, usage_pct
        else:
            return 0, 0, 0

    def on_train_begin(self, args, state, control, **kwargs):
        run_name = f"wav2vec2"
        wandb.init(
            project="DNE_baseline", ######################################################################################################
            name=run_name,
            config=args.to_dict()
        )
        print(f"Logging W&B bắt đầu | Thiết bị: {self.gpu_name}")

    def on_epoch_end(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            epoch = int(state.epoch)
            gpu_mem_used, gpu_mem_total, gpu_usage_pct = self.get_gpu_stats()
            wandb.log({
                "epoch": epoch,
                "train/loss": metrics.get("loss"),
                "eval/loss": metrics.get("eval_loss"),
                "eval/accuracy": metrics.get("eval_accuracy"),
                "eval/f1": metrics.get("eval_f1"),
                "gpu/name": self.gpu_name,
                "gpu/memory_used_GB": gpu_mem_used,
                "gpu/memory_total_GB": gpu_mem_total,
                "gpu/memory_usage_%": gpu_usage_pct,
            })
            print(f"Epoch {epoch}: "
                  f"F1={metrics.get('eval_f1'):.4f}, "
                  f"Acc={metrics.get('eval_accuracy'):.4f}, "
                  f"GPU={gpu_usage_pct:.1f}% VRAM ({gpu_mem_used:.2f}/{gpu_mem_total:.2f} GB)")

    def on_save(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        default_ckpt = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        new_ckpt = os.path.join(args.output_dir, f"checkpoint-epoch-{epoch:02d}")
        if os.path.exists(default_ckpt):
            try:
                os.rename(default_ckpt, new_ckpt)
                # print(f"Đổi tên checkpoint thành: {new_ckpt}")
                wandb.log({"checkpoint_saved_epoch": epoch})
            except Exception as e:
                print(f"Không thể đổi tên checkpoint: {e}")

    def on_train_end(self, args, state, control, **kwargs):
        wandb.finish()
        print("xong")


In [9]:
from transformers import TrainingArguments, Trainer
from transformers.trainer_callback import TrainerCallback


OUTPUT_DIR = "D:/AIP491/DNE/check_points_baseline/wav2vec2"
EPOCHS = 5 
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=EPOCHS,
    gradient_accumulation_steps=2,
    warmup_ratio=0.05,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_steps= 500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    disable_tqdm=False,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    callbacks=[WandbEpochLogger()]
)


In [10]:
import pandas as pd
link_csv_train = pd.read_csv('D:\\AIP491\\DNE\\csv\\DNE_train.csv')
print('train:',link_csv_train['label'].unique())
print(link_csv_train['label'].value_counts())
link_csv_val = pd.read_csv('D:\\AIP491\\DNE\\csv\\DNE_val.csv')
print('val:',link_csv_val['label'].unique())
print(link_csv_val['label'].value_counts())
link_csv_test = pd.read_csv('D:\\AIP491\\DNE\\csv\\DNE_test.csv')
print('test', link_csv_test['label'].unique())
print(link_csv_test['label'].value_counts())

train: ['pos' 'neg']
label
pos    11222
neg     7713
Name: count, dtype: int64
val: ['neg' 'pos']
label
pos    1550
neg    1475
Name: count, dtype: int64
test ['pos' 'neg']
label
pos    2703
neg    1822
Name: count, dtype: int64


In [11]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)

print(f"Total parameters: {total:,}")
print(f"Trainable parameters: {trainable:,}")

Total parameters: 315,701,634
Trainable parameters: 315,701,634


In [14]:
trainer.train()
# # nvidia-smi -l 30

wandb: Currently logged in as: zidanepm (zidanepm-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Logging W&B bắt đầu | Thiết bị: NVIDIA GeForce RTX 3050 Laptop GPU


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.560700,0.573850,0.708430,0.758514,0.632258,0.689655
2,0.511000,0.554936,0.730909,0.788401,0.649032,0.711960
3,0.531000,0.597698,0.729256,0.800329,0.628387,0.704012
4,0.484600,0.546488,0.764959,0.795215,0.729032,0.760687
5,0.451700,0.533582,0.771570,0.815114,0.716774,0.762788


checkpoint_saved_epoch,▁▃▅▆█
eval/accuracy,▁▃▃▇█
eval/f1,▁▃▂██
eval/loss,▅▃█▂▁
eval/precision,▁▅▆▆█
eval/recall,▁▂▁█▇
eval/runtime,▁▆▂█▃
eval/samples_per_second,█▃▇▁▆
eval/steps_per_second,█▃█▁█
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
+4,...


xong


TrainOutput(global_step=11835, training_loss=0.5123876042239113, metrics={'train_runtime': 373621.9648, 'train_samples_per_second': 0.253, 'train_steps_per_second': 0.032, 'total_flos': 2.29547920887936e+19, 'train_loss': 0.5123876042239113, 'epoch': 5.0})

In [ ]:
import subprocess

subprocess.run([
    "wandb", "sync",
    r"D:\\AIP491\\DNE\\code\\baseline\\wav2vec2-xls-r-300m\\wandb\\run-20251223_221443-ue0r66hd"
], check=True)


In [13]:
# import os, time
# time.sleep(300)
# os.system('shutdown /s /t 0')